In [13]:
import pandas as pd

df_happy = pd.read_csv('../../../../2019happy.csv')

df_WDI = pd.read_csv('../../../../WDI.csv')


In [16]:
print("Happiness DF columns:\n", df_happy.columns.tolist())
print("\nWDI DF columns:\n", df_WDI.columns.tolist())


Happiness DF columns:
 ['Overall rank', 'Country or region', 'Score', 'GDP per capita', 'Social support', 'Healthy life expectancy', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption']

WDI DF columns:
 ['Country Name', 'Country Code', 'Series Name', 'Series Code', '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]', '2019 [YR2019]']


In [ ]:
# white space weghalen indien nodig
df_WDI_long['Series_Code'] = df_WDI_long['Series_Code'].str.strip()

# 2. de code een normale naam geven
wanted_codes = {
    'NY.GDP.PCAP.KN':     'GDP_per_capita_const_LCU',
    'NY.GDY.TOTL.KN':     'Gross_income_const_LCU',
    'SI.POV.GINI':        'Gini_index',
    'SE.XPD.TOTL.GD.ZS':  'Govt_educ_pct_GDP'
}


df_WDI_filt = df_WDI_long[df_WDI_long['Series_Code'].isin(wanted_codes)]
df_WDI_wide = (
    df_WDI_filt
      .pivot_table(
         index=['Country','Year'],
         columns='Series_Code',
         values='Value',
         aggfunc='first'
      )
      .rename(columns=wanted_codes)
      .reset_index()
)

# 4. Merge met 2019happy
df = df_happy.merge(df_WDI_wide, on=['Country','Year'], how='inner')
print(df.shape)
df.head()


(132, 14)


,Overall rank,Country,Happiness,GDP_per_capita,Social_support,Healthy_life_expectancy,Freedom_life_choices,Generosity,Perceptions_corruption,Year,GDP_per_capita_const_LCU,Gross_income_const_LCU,Govt_educ_pct_GDP,Gini_index
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,2019,4.136822e+04,2.279321e+11,6.41695,27.7
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,2019,3.842619e+05,2.226293e+12,7.25505,27.7
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,2019,6.186207e+05,3.252660e+12,7.86456,27.7
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,2019,7.584458e+06,2.727708e+12,7.60130,NaN
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298,2019,4.439247e+04,7.773208e+11,5.15986,29.2


In [27]:
df.to_csv("../../../../merge_happy_wdi.csv", index=False)
